# Semantic SQL Tutorial

This tutorial assumes you have read introductory material on
https://github.com/INCATools/semantic-sql




## Part 1: Using SQLite3 directly

This part of the tutorial walks through queries using the SQLite interface. No python is required.

You can follow along in two ways

1. By running this notebook interactively
2. By running the commands directly on the command line

If you want to follow the second route, you will need to first install sqlite. After that, you can execute
any command in this notebook that starts with `%` using the sqlite command line tool.

### Download pre-made Cell Ontology db file

We will be making use of the [Cell Ontology](https://obofoundry.org/ontology/cl).

First we will download the ready-made cl.db file from the SemSQL repository:

In [1]:
!wget https://s3.amazonaws.com/bbop-sqlite/cl.db -O cl.db

--2022-08-15 15:45:49--  https://s3.amazonaws.com/bbop-sqlite/cl.db
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.17.195
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.17.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 661434368 (631M) [binary/octet-stream]
Saving to: ‘cl.db’

cl.db               100%[===================>] 630.79M  4.23MB/s    in 1m 58s  

2022-08-15 15:47:48 (5.34 MB/s) - ‘cl.db’ saved [661434368/661434368]



This is quite large so it make take a few minutes depending on your connection.

next, some Jupyter magic to connect to cl.db

if you are trying this from the command line, instead do this:

```
sqlite cl.db
```

In [2]:
%load_ext sql

In [3]:
%sql sqlite:///cl.db

### Exploring CL via labels

The [rdfs_label_statement](https://incatools.github.io/semantic-sql/RdfsLabelStatement/) view provides a way to query triples that use rdfs:label.
    
To get a sample of terms in CL that have a label with `neuron` in it:

In [4]:
%sql SELECT * FROM rdfs_label_statement WHERE value LIKE '%neuron%' LIMIT 10

 * sqlite:///cl.db
Done.


stanza,subject,predicate,object,value,datatype,language
RO:0002132,RO:0002132,rdfs:label,None,has fasciculating neuron projection,None,en
CARO:0001001,CARO:0001001,rdfs:label,None,neuron projection bundle,None,None
CL:0000006,CL:0000006,rdfs:label,None,neuronal receptor cell,xsd:string,None
CL:0000028,CL:0000028,rdfs:label,None,CNS neuron (sensu Nematoda and Protostomia),xsd:string,None
CL:0000029,CL:0000029,rdfs:label,None,neural crest derived neuron,None,None
CL:0000047,CL:0000047,rdfs:label,None,neuronal stem cell,xsd:string,None
CL:0000095,CL:0000095,rdfs:label,None,neuron associated cell,xsd:string,None
CL:0000099,CL:0000099,rdfs:label,None,interneuron,xsd:string,None
CL:0000100,CL:0000100,rdfs:label,None,motor neuron,xsd:string,None
CL:0000101,CL:0000101,rdfs:label,None,sensory neuron,xsd:string,None


You can get an explanation of each column on the page for [rdfs_label_statement](https://incatools.github.io/semantic-sql/RdfsLabelStatement/). Each table and column should have complete definitions and metadata.

Note that some columns can be ignored. `object` is always None for rdfs_label_statement. We'll see an explanation later

### Querying all triples

Next we will query all triples for "enteric neuron" [CL:0007011](http://purl.obolibrary.org/obo/CL_0007011)

To do this we will use the [statements](https://incatools.github.io/semantic-sql/Statements/) base table:

In [7]:
%sql SELECT * FROM statements WHERE subject='CL:0007011'

 * sqlite:///cl.db
Done.


stanza,subject,predicate,object,value,datatype,language
CL:0007011,CL:0007011,IAO:0000115,None,Neuron that is part of the enteric nervous system.,xsd:string,None
CL:0007011,CL:0007011,oio:created_by,None,haendel,xsd:string,None
CL:0007011,CL:0007011,oio:creation_date,None,2012-06-28T08:20:31Z,xsd:string,None
CL:0007011,CL:0007011,owl:equivalentClass,_:riog00020174,None,None,None
CL:0007011,CL:0007011,rdf:type,owl:Class,None,None,None
CL:0007011,CL:0007011,rdfs:label,None,enteric neuron,xsd:string,None
CL:0007011,CL:0007011,rdfs:subClassOf,_:riog00020181,None,None,None
CL:0007011,CL:0007011,rdfs:subClassOf,_:riog00020180,None,None,None
CL:0007011,CL:0007011,rdfs:subClassOf,_:riog00020179,None,None,None
CL:0007011,CL:0007011,rdfs:subClassOf,CL:0000107,None,None,None


You will note that this includes odd entries such as `_riog1234`. This is how *blank nodes* are stored in the underlying SQL database.

### Views provide convenient abstractions

If you are looking at blank nodes, it's an indication that you are working **at too low level of abstraction**. The statements table is the "foundational" layer of SemSQL, but unlike with a triplestore, there are ready-made abstractions.

The previous [rdfs_label_statement](https://incatools.github.io/semantic-sql/RdfsLabelStatement/) view we used is an example of this. This view is defined as:

```
CREATE VIEW rdfs_label_statement AS 
  SELECT * FROM statements
  WHERE predicate='rdfs:label'
```

Views are *composable* ultimately allowing you to create higher level abstractions

### Graph structure: Edges

Next we will use the [edge](https://incatools.github.io/semantic-sql/Edge/) view to find all outgoing edges from a node.

(Note this is actually quite hard to do as an equivalent SPARQL query, since the way OWL stores edges in RDF is quite abstract, using lots of blank nodes)

An edge is simply a subject-predicate-object triple. Once again we'll query enteric neuron:

In [6]:
%sql SELECT * FROM edge WHERE subject='CL:0007011'

 * sqlite:///cl.db
Done.


subject,predicate,object
CL:0007011,BFO:0000050,UBERON:0002005
CL:0007011,RO:0002100,UBERON:0002005
CL:0007011,RO:0002202,CL:0002607
CL:0007011,rdfs:subClassOf,CL:0000029
CL:0007011,rdfs:subClassOf,CL:0000107


No blank nodes!

Next let's make the results a bit more informative by doing a join with the label table:

In [6]:
%%sql
SELECT DISTINCT edge.*, pl.value AS pred_label, ol.value AS object_label
FROM
    edge
    LEFT JOIN rdfs_label_statement AS pl ON (edge.predicate=pl.subject)
    INNER JOIN rdfs_label_statement AS ol ON (edge.object=ol.subject)
WHERE 
    edge.subject='CL:0007011'


 * sqlite:///cl.db
Done.


subject,predicate,object,pred_label,object_label
CL:0007011,BFO:0000050,UBERON:0002005,part_of,enteric nervous system
CL:0007011,BFO:0000050,UBERON:0002005,part of,enteric nervous system
CL:0007011,RO:0002100,UBERON:0002005,has soma location,enteric nervous system
CL:0007011,RO:0002202,CL:0002607,develops_from,migratory enteric neural crest cell
CL:0007011,RO:0002202,CL:0002607,develops from,migratory enteric neural crest cell
CL:0007011,rdfs:subClassOf,CL:0000029,None,neural crest derived neuron
CL:0007011,rdfs:subClassOf,CL:0000107,None,autonomic neuron


Note we use a LEFT JOIN for the predicate label, as not all predicates are guaranteed to have a label
(particularly builtin ones like rdfs:subClassOf).

Unfortunately SQL syntax is a little more verbose than SPARQL here - I find having to write
explicit JOIN statements tedious. But one advantage of SQL is that if you find yourself writing
the same patterns again and again you can create a VIEW to capture this!

### Entailed edges (ancestors)

All SemSQL databases have an ancestor table pre-cached. 

This is called [entailed_edge](https://incatools.github.io/semantic-sql/EntailedEdge/). It is more sophisticated than
a simple transitive closure over the graph. A row is only present in entailed_edge if that edge is *entailed* (inferrable from base axioms) in the ontology.

Note this means that you don't need to make any assumptions about the ontology being a DAG. these assumptions are usually false! entailed_edge is guaranteed to give the correct answer. Under the hood, this is built using [relation-graph](https://github.com/balhoff/relation-graph/)

In [11]:
%sql SELECT * FROM entailed_edge WHERE subject='CL:0007011' LIMIT 20

 * sqlite:///cl.db
Done.


subject,predicate,object
CL:0007011,BFO:0000050,BFO:0000002
CL:0007011,BFO:0000050,BFO:0000004
CL:0007011,BFO:0000050,BFO:0000040
CL:0007011,BFO:0000050,CARO:0000000
CL:0007011,BFO:0000050,CARO:0000006
CL:0007011,BFO:0000050,CARO:0030000
CL:0007011,BFO:0000050,UBERON:0000010
CL:0007011,BFO:0000050,UBERON:0000061
CL:0007011,BFO:0000050,UBERON:0000465
CL:0007011,BFO:0000050,UBERON:0000467


### Complex queries

One advantage of a query layer such as SQL or SPARQL is being able to write arbitrarily complex queries in a declarative fashion.

Let's explore a query over CL to fetch all neurons that are part of the forebrain

In [40]:
%%sql 
SELECT DISTINCT lbl.subject, lbl.value 
FROM 
  entailed_edge AS e_neuron,
  entailed_edge AS e_forebrain,
  rdfs_label_statement AS lbl
WHERE 
  e_neuron.object='CL:0000540' AND
  e_forebrain.object='UBERON:0001890' AND
  e_neuron.predicate = 'rdfs:subClassOf' AND
  e_forebrain.predicate = 'RO:0002100' AND
  e_neuron.subject=e_forebrain.subject AND
  e_neuron.subject = lbl.subject
LIMIT 20    

 * sqlite:///cl.db
Done.


subject,value
CL:0000626,olfactory granule cell
CL:0001032,cortical granule cell
CL:0001033,hippocampal granule cell
CL:0002608,hippocampal neuron
CL:0002613,striatum neuron
CL:0008031,cortical interneuron
CL:0008032,rosehip neuron
CL:0008049,Betz cell
CL:0010011,cerebral cortex GABAergic interneuron
CL:0010012,cerebral cortex neuron


## Part 2: Using Python and SQL Alchemy

Next we will explore programmatic usage using Python. If you are an experienced developer it should be possible
for you to see how to adapt this to your language of choice.

This part is in two sub-parts:

- 2.1: direct SQL queries (no ORM)
- 2.2: using the SemSQL ORM

### 2.1 Direct SQL queries

First we will make a SQL Alchemy connection

In [7]:
from sqlalchemy import create_engine

In [10]:
# next we will create an engine and connection
engine = create_engine(f"sqlite:///cl.db")
conn = engine.connect()

In [19]:
# first example: hardcode the term of interest
result = conn.execute("SELECT * FROM edge WHERE subject='CL:0007011'")

In [12]:
for row in result:
    print(row)

('CL:0007011', 'BFO:0000050', 'UBERON:0002005')
('CL:0007011', 'RO:0002100', 'UBERON:0002005')
('CL:0007011', 'RO:0002202', 'CL:0002607')
('CL:0007011', 'rdfs:subClassOf', 'CL:0000029')
('CL:0007011', 'rdfs:subClassOf', 'CL:0000107')


#### Parameterizing queries using text()

For more background see [using textual sql](https://docs.sqlalchemy.org/en/14/core/tutorial.html#using-textual-sql)

In [13]:
from sqlalchemy.sql import text

In [14]:
edge_query = text("SELECT * FROM edge WHERE subject=:subject")

In [16]:
for row in conn.execute(edge_query, subject='CL:0007011'):
    print(row)

('CL:0007011', 'BFO:0000050', 'UBERON:0002005')
('CL:0007011', 'RO:0002100', 'UBERON:0002005')
('CL:0007011', 'RO:0002202', 'CL:0002607')
('CL:0007011', 'rdfs:subClassOf', 'CL:0000029')
('CL:0007011', 'rdfs:subClassOf', 'CL:0000107')


### 2.2 Using the SemSQL ORM

An ORM (Object-Relational Mapper) can help bridge the gap between logic in a language like Python and a SQL database.

ORMs are divisive: some people love them, some hate them! Like most tools they have advantages and disadvantages.
For our purposes, ORMs can better expose your underlying data model so you can get typing and autocomplete
assistance from your IDE (or here, in Jupyter).

However, you might want to avoid ORMs for scenarios where performance is key.

We will give an example of doing things using the SQL Alchemy ORM using SemSQL ORM classes.

This tutorial doesn't substitute for a full-blown SQL Alahcmey tutorial. Please consult https://docs.sqlalchemy.org/en/14/orm/ for more docs

Let's get started by importing the relevant classes and methods, from SQLA and from SemSQL:

In [22]:
from sqlalchemy.orm import sessionmaker, aliased
from semsql.sqla.semsql import *

In [23]:
# create an ORM session
SessionClass = sessionmaker(bind=engine)
session = SessionClass()

#### Querying edges with the ORM

Next we will do the equivalent of the edge query above, but this time using the Edge class in the ORM.

In [27]:
for row in session.query(Edge).filter(Edge.subject=='CL:0007011'):
    print(row)

edge(subject=CL:0007011,predicate=BFO:0000050,object=UBERON:0002005,)
edge(subject=CL:0007011,predicate=RO:0002100,object=UBERON:0002005,)
edge(subject=CL:0007011,predicate=RO:0002202,object=CL:0002607,)
edge(subject=CL:0007011,predicate=rdfs:subClassOf,object=CL:0000029,)
edge(subject=CL:0007011,predicate=rdfs:subClassOf,object=CL:0000107,)


#### Query composition with the ORM

One advantage of the ORM is being able to compose together query logic and avoiding SQL string concatenation

In [33]:
for edge, lbl in session.query(Edge, RdfsLabelStatement).join(RdfsLabelStatement, RdfsLabelStatement.subject==Edge.object).filter(Edge.subject=='CL:0007011'):
    print(f"{edge.predicate} {edge.object} {lbl.value}")

rdfs:subClassOf CL:0000029 neural crest derived neuron
rdfs:subClassOf CL:0000107 autonomic neuron
RO:0002202 CL:0002607 migratory enteric neural crest cell
BFO:0000050 UBERON:0002005 enteric nervous system
RO:0002100 UBERON:0002005 enteric nervous system


That's maybe getting quite verbose for one line, so we can break it down:


In [35]:
q = session.query(Edge, RdfsLabelStatement)
q = q.join(RdfsLabelStatement, RdfsLabelStatement.subject==Edge.object)
q = q.filter(Edge.subject=='CL:0007011')
 
for edge, lbl in q:
    print(f"{edge.predicate} {edge.object} {lbl.value}")

rdfs:subClassOf CL:0000029 neural crest derived neuron
rdfs:subClassOf CL:0000107 autonomic neuron
RO:0002202 CL:0002607 migratory enteric neural crest cell
BFO:0000050 UBERON:0002005 enteric nervous system
RO:0002100 UBERON:0002005 enteric nervous system


SQLA uses the builder pattern for queries, allowing us to keep composing new joins and constraints.

Here we can see how to build up a complex query programmatically.

### Introspecting ORM classes

These can be introspected in Jupyter or in your IDE

Currently this is a bit verbose...

In [28]:
help(Edge)

Help on class Edge in module semsql.sqla.semsql:

class Edge(RelationGraphConstruct)
 |  Edge(**kwargs)
 |  
 |  A relation graph edge that connects two entities by a predicate. Note an edge is distinct from a statement, in that an axiom such as A SubClassOf R some B is represented as multiple statements, but is a single relation graph edge
 |  
 |  Method resolution order:
 |      Edge
 |      RelationGraphConstruct
 |      sqlalchemy.orm.decl_api.Base
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, **kwargs)
 |      A simple constructor that allows initialization from kwargs.
 |      
 |      Sets attributes on the constructed instance using the names and
 |      values in ``kwargs``.
 |      
 |      Only keys that are present as
 |      attributes of the instance's class are allowed. These could be,
 |      for example, any mapped columns or relationships.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  ------------------------------------

## Next Steps

At the end of this tutorial you should have an idea of

- how to download a SQLite database from the SemSQL repo
- the basics of how tables and views are organized in a SemSQL sqlite database
- how to do basic queries using the SQLite command line interface
- how to find out more about existing views and tables
- how to access SemSQL database programmatically via Python
- how to use the ORM

For many purposes it might not be necessary to write your own code

[OAK](https://incatools.github.io/ontology-access-kit) provides an abstraction layer on top of different endpoints, including Semantic-SQL. For some operations it may be better querying via an OAK interface

See also:

https://incatools.github.io/ontology-access-kit/intro/tutorial07.html